## Черевичин Егор М8О-406Б-21 Лабораторная работа 7

In [1]:
import os
import random

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import segmentation_models_pytorch as smp
from torchvision.datasets import VOCSegmentation

from torchmetrics.classification import MulticlassJaccardIndex

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 21

# Обоснования выбора VOC2012 
1. Общеизвестный бенчмарк
VOC2012 — классический датасет, широко используемый в задачах семантической сегментации. Он позволяет объективно сравнивать модели с результатами других работ.

2. Разнообразие классов
Включает 20 классов объектов и фон — охватывает широкий спектр категорий (животные, транспорт, люди и т.д.), что делает его универсальным для оценки сегментаторов.

3. Качественная разметка
Все маски размечены вручную, обеспечивая точные и надёжные аннотации.

4. Умеренная сложность
Сцены содержат частично перекрывающиеся объекты и разнообразные формы, что делает датасет достаточно сложным для обучения и тестирования моделей.

5. Поддержка в популярных фреймворках
VOC2012 встроен в PyTorch, TensorFlow и другие библиотеки, что упрощает его использование и интеграцию в пайплайны.

### Подготовка датасета

#### Аугментации (Albumentations)

| Режим       | Описание                                                                                      |
|-------------|-----------------------------------------------------------------------------------------------|
| **train**   | - Случайный масштаб и кроп до 512×512  <br> - Горизонтальный флип  <br> - Color jitter        |
|             | - Нормализация по статистике ImageNet  <br> - Преобразование в тензор                         |
| **val**     | - Масштаб по длинной стороне до 512 px  <br> - Паддинг до 512×512                             |
|             | - Нормализация как в train  <br> - Преобразование в тензор                                    |

#### Класс `VOCDataset`

| Компонент         | Описание                                                                                  |
|-------------------|-------------------------------------------------------------------------------------------|
| Источник          | `torchvision.datasets.VOCSegmentation(year=2012)`                                         |
| `__getitem__`     | - Конвертация изображений и масок из PIL в NumPy <br> - Совместные аугментации пары      |
|                   | - Возврат: изображение (`float`), маска (`long`)                                          |

#### DataLoader

| Назначение  | batch_size | shuffle | pin_memory | num_workers | Комментарий                              |
|-------------|------------|---------|------------|-------------|------------------------------------------|
| **train**   | 8          | True    | True       | 4           | Быстрая загрузка и случайное перемешивание |
| **val**     | 8          | False   | True       | 4           | Детерминированная оценка                  |



In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import VOCSegmentation

import albumentations as A
from albumentations.pytorch import ToTensorV2

train_tfms = A.Compose([
    A.RandomResizedCrop(size=(512, 512), scale=(0.5, 1.0), ratio=(0.75, 1.33), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(0.2, 0.2, 0.2, 0.1, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_tfms = A.Compose([
    A.LongestMaxSize(512),
    A.PadIfNeeded(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

class VOCDataset(torch.utils.data.Dataset):
    def __init__(self, root, image_set, tfms):
        self.voc = VOCSegmentation(
            root=root,
            year="2012",
            image_set=image_set,
            download=True,
        )
        self.tfms = tfms

    def __len__(self):
        return len(self.voc)

    def __getitem__(self, idx):
        img, mask = self.voc[idx]
        aug = self.tfms(image=np.array(img), mask=np.array(mask))
        image = aug["image"]
        mask = torch.as_tensor(aug["mask"], dtype=torch.long)
        return image, mask

ds_train = VOCDataset(
    root="./data",
    image_set="train",
    tfms=train_tfms,
)
ds_val = VOCDataset(
    root="./data",
    image_set="val",
    tfms=val_tfms,
)

dl_train = DataLoader(
    ds_train,
    batch_size=8,
    shuffle=True,
    pin_memory=True,
    num_workers=4,
)
dl_val = DataLoader(
    ds_val,
    batch_size=8,
    shuffle=False,
    pin_memory=True,
    num_workers=4,
)

100%|██████████| 1999639040/1999639040 [00:36<00:00, 55104474.69it/s]


Extracting ./data/VOCtrainval_11-May-2012.tar to ./data
Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data


## Метрики и общий цикл обучения

### Функции потерь и метрики

| Название                         | Описание                                                                                   | Особенности                                           |
|----------------------------------|--------------------------------------------------------------------------------------------|--------------------------------------------------------|
| **CrossEntropy + DiceLoss**      | Комбинация двух функций потерь:                                                            | 70% CrossEntropy + 30% DiceLoss                       |
|                                  | - CrossEntropy для точной пиксельной классификации                                         | Игнорируются пиксели с меткой `255` (нет разметки)   |
|                                  | - Dice Loss для улучшения перекрытия предсказанных и истинных масок                       |                                                      |
| **IoU (Intersection over Union)**| Оценка качества сегментации — отношение площади пересечения к объединению сегментов       | Вычисляется по каждому классу                         |
| **mIoU (mean IoU)**              | Среднее значение IoU по всем классам                                                       | Ключевая метрика, исключает метку `255`              |
| **Dice Loss**                    | Измеряет степень перекрытия, чувствителен к малым сегментам                                | Часто используется в дополнение к CrossEntropy        |

---

#### Визуальное сравнение IoU и Dice (схематично)

- **IoU** = |A ∩ B| / |A ∪ B|  
- **Dice** = 2 × |A ∩ B| / (|A| + |B|)

Где:
- A — предсказанная маска  
- B — истинная маска

> Dice более чувствителен к небольшим объектам и может лучше работать при несбалансированных классах.



In [3]:
IGNORE = 255

def get_metrics():
    return MulticlassJaccardIndex(
        num_classes=NUM_CLASSES,
        ignore_index=IGNORE,
    ).to(DEVICE)

dice_loss = smp.losses.DiceLoss(
    mode="multiclass",
    ignore_index=IGNORE,
    smooth=1e-5,
)
ce_loss = nn.CrossEntropyLoss(ignore_index=IGNORE)

def loss_fn(logits, targets):
    return 0.7 * ce_loss(logits, targets) + 0.3 * dice_loss(logits, targets)

def train_one_epoch(model, opt, scaler):
    model.train()
    for x, y in dl_train:
        x, y = x.to(DEVICE), y.to(DEVICE)
        opt.zero_grad()
        with torch.cuda.amp.autocast():
            loss = loss_fn(model(x), y)
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()

@torch.no_grad()
def evaluate(model, metric):
    model.eval(); metric.reset()
    for x, y in dl_val:
        x, y = x.to(DEVICE), y.to(DEVICE)
        logits = model(x).argmax(1)
        metric.update(preds=logits, target=y)
    return metric.compute().item()

def run_training(model, epochs=10, lr=1e-3):
    model.to(DEVICE)
    opt = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=1e-2,
    )
    sch = torch.optim.lr_scheduler.LambdaLR(
        opt,
        lambda e: (1 - e / epochs) ** 0.9,
    )
    scaler = torch.cuda.amp.GradScaler()
    metric = get_metrics()

    for ep in range(epochs):
        train_one_epoch(model, opt, scaler)
        miou = evaluate(model, metric)
        print(f"E{ep:02d}: mIoU={miou:.3f}")
        sch.step()

    return miou

## Baseline модели

### Обычная ResNet

Baseline ResNet34-U-Net

- **Архитектура**: U-Net с энкодером ResNet34 (предобучен на ImageNet), 21 класс сегментации.  
- **Тренинг**: 20 эпох, lr=1e-3, оптимизатор AdamW, смешанная точность, LambdaLR-шедулер.  

In [12]:
baseline_cnn = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    classes=NUM_CLASSES,
).to(DEVICE)

miou_base_cnn = run_training(
    model=baseline_cnn,
    epochs=20,
    lr=1e-3,
)

print(f"Baseline-CNN mIoU: {miou_base_cnn:.3f}")

E00: mIoU=0.039
E01: mIoU=0.043
E02: mIoU=0.047
E03: mIoU=0.057
E04: mIoU=0.079
E05: mIoU=0.071
E06: mIoU=0.073
E07: mIoU=0.082
E08: mIoU=0.099
E09: mIoU=0.085
E10: mIoU=0.095
E11: mIoU=0.108
E12: mIoU=0.086
E13: mIoU=0.104
E14: mIoU=0.115
E15: mIoU=0.117
E16: mIoU=0.127
E17: mIoU=0.120
E18: mIoU=0.124
E19: mIoU=0.134
Baseline-CNN mIoU: 0.134


### Transformer

### Обучение модели

- **Модель:**  
  Используется архитектура SegFormer с энкодером **MiT-B0**, предварительно обученным на ImageNet.

- **Процесс обучения:**  
  Запуск функции: `run_training(model, epochs=20, lr=1e-4)`

  Параметры:
  - Количество эпох: **20**
  - Начальная скорость обучения: **1e-4**
  - Оптимизатор: **AdamW**
  - Планировщик обучения: **экспоненциальный LR scheduler**
  - Активирован **AMP (смешанная точность)** для ускорения и экономии памяти
  - Основная метрика: **mIoU** (mean Intersection over Union) на валидационном наборе


In [13]:
baseline_trans = smp.Segformer(
    encoder_name="mit_b0",
    encoder_weights="imagenet",
    classes=NUM_CLASSES,
).to(DEVICE)

miou_base_trans = run_training(
    model=baseline_trans,
    epochs=20,
    lr=1e-4,
)

print(f"Baseline-Transformer mIoU: {miou_base_trans:.3f}")

E00: mIoU=0.404
E01: mIoU=0.518
E02: mIoU=0.535
E03: mIoU=0.566
E04: mIoU=0.579
E05: mIoU=0.569
E06: mIoU=0.598
E07: mIoU=0.579
E08: mIoU=0.613
E09: mIoU=0.626
E10: mIoU=0.600
E11: mIoU=0.624
E12: mIoU=0.636
E13: mIoU=0.637
E14: mIoU=0.632
E15: mIoU=0.643
E16: mIoU=0.656
E17: mIoU=0.653
E18: mIoU=0.650
E19: mIoU=0.650
Baseline-Transformer mIoU: 0.650


### Динамика обучения

- **Начальная фаза (эпохи 0–3):**  
  Резкий рост mIoU — от 0.404 до 0.566. Модель быстро учится выделять основные объекты на изображениях.

- **Средняя фаза (эпохи 4–11):**  
  Прирост становится более умеренным: mIoU повышается с 0.579 до 0.624. Видны небольшие колебания, но тренд остаётся положительным.

- **Поздняя фаза (эпохи 12–19):**  
  Модель выходит на плато — метрика достигает 0.650. Улучшения становятся минимальными, что указывает на стабилизацию обучения.

---

- **Итоговая mIoU после 20 эпох:** `0.650`  
- **Общее поведение:**  
  Значительное улучшение наблюдается уже в первые 3–4 эпохи, далее рост замедляется и постепенно выходит на насыщение.

> Архитектура SegFormer (MiT-B0) заметно превосходит U-Net с ResNet34 — достигает более высокой точности сегментации и быстрее сходится при обучении.


## Улучшенные бейзлайны

### Улучшенный бейзлайн обычной

**Архитектура:** U-Net++ с энкодером EfficientNet-B5 (ImageNet) и SCSE-вниманием в декодере.  
**Данные:** VOC2012 (train/val).  
**Тренинг:** epochs=20, lr=5·10⁻⁴, AdamW, комбинированный loss (CE + Dice).

In [6]:
imp_cnn = smp.UnetPlusPlus(
    encoder_name="tu-efficientnet_b5",
    encoder_weights="imagenet",
    classes=NUM_CLASSES,
    decoder_attention_type="scse",
).to(DEVICE)

miou_imp_cnn = run_training(
    model=imp_cnn,
    epochs=10,
    lr=5e-4,
)

print(f"Improved-CNN mIoU: {miou_imp_cnn:.3f}")

Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


E00: mIoU=0.053
E01: mIoU=0.075
E02: mIoU=0.086
E03: mIoU=0.097
E04: mIoU=0.118
E05: mIoU=0.139
E06: mIoU=0.150
E07: mIoU=0.176
E08: mIoU=0.156
E09: mIoU=0.181
E10: mIoU=0.184
E11: mIoU=0.168
E12: mIoU=0.160
E13: mIoU=0.216
E14: mIoU=0.207
E15: mIoU=0.239
E16: mIoU=0.251
E17: mIoU=0.264
E18: mIoU=0.266
E19: mIoU=0.257
Improved-CNN mIoU: 0.257


### Улучшенный трансформер

### Обучение улучшенной трансформерной модели

- **Модель:**  
  Используется SegFormer с энкодером **MiT-B2** — более глубокая и мощная версия по сравнению с MiT-B0.  
  Настройка декодера: `decoder_segmentation_channels=512`.  
  Весы энкодера и декодера инициализированы из модели, предобученной на ImageNet.

- **Процесс обучения:**  
  Вызов: `run_training(model=imp_trans, epochs=20, lr=1.5e-4)`

  Параметры обучения:
  - Эпохи: **20**
  - Начальное значение learning rate: **1.5e-4**
  - Оптимизатор: **AdamW**
  - Планировщик: **LambdaLR** — позволяет гибко управлять спадом lr
  - Используется **смешанная точность (AMP)** для ускорения и экономии памяти
  - Ключевая метрика: **mIoU** (среднее IoU по классам) на валидационном наборе


In [7]:
imp_trans = smp.Segformer(
    encoder_name="mit_b2",
    encoder_weights="imagenet",
    classes=NUM_CLASSES,
    decoder_segmentation_channels=512,
).to(DEVICE)

miou_imp_trans = run_training(
    model=imp_trans,
    epochs=20,
    lr=1.5e-4,
)

print(f"Improved-Transformer mIoU: {miou_imp_trans:.3f}")

E00: mIoU=0.263
E01: mIoU=0.374
E02: mIoU=0.419
E03: mIoU=0.353
E04: mIoU=0.428
E05: mIoU=0.491
E06: mIoU=0.485
E07: mIoU=0.563
E08: mIoU=0.541
E09: mIoU=0.585
E10: mIoU=0.596
E11: mIoU=0.581
E12: mIoU=0.632
E13: mIoU=0.578
E14: mIoU=0.636
E15: mIoU=0.645
E16: mIoU=0.671
E17: mIoU=0.672
E18: mIoU=0.674
E19: mIoU=0.684
Improved-Transformer mIoU: 0.684


### Результаты обучения

- **Стартовая точка:**  
  Обучение началось с mIoU = 0.263 на первой эпохе (E00).

- **Лучший результат:**  
  К 19-й эпохе (E19) модель достигла максимального значения mIoU = 0.684.

- **Динамика роста:**  
  Существенный прирост начался после 6-й эпохи, затем наблюдался стабильный прогресс вплоть до завершения обучения.  
  Финальное значение mIoU (0.684) превышает результат базовой версии трансформера (0.650).

- **Вывод:**  
  Более глубокий энкодер **MiT-B2** позволяет модели эффективнее захватывать контекст и пространственные зависимости, что напрямую отражается на улучшении качества сегментации по сравнению с MiT-B0.


## Свои модели

### Обычная

### Обучение собственной CNN-модели

- **Архитектура:**  
  Лёгкий вариант U-Net с четырьмя уровнями энкодера и декодера.  
  Начальная ширина каналов — **32**. Каждый блок состоит из последовательности: **Conv(3×3) → BatchNorm → ReLU**.  
  Для апсемплинга используется **ConvTranspose2d**.

- **Обучение:**  
  Модель обучалась **12 эпох** с начальными параметрами:  
  - Learning rate: **1e-3**


In [8]:
def CBR(in_c: int, out_c: int) -> nn.Sequential:
    return nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True),
    )


class Down(nn.Module):
    def __init__(self, in_c: int, out_c: int):
        super().__init__()
        self.seq = nn.Sequential(
            CBR(in_c, out_c),
            CBR(out_c, out_c),
        )

    def forward(self, x):
        return self.seq(x)


class Up(nn.Module):
    def __init__(self, in_c: int, out_c: int):
        super().__init__()
        self.up = nn.ConvTranspose2d(
            in_c,
            in_c // 2,
            kernel_size=2,
            stride=2,
        )
        self.conv = nn.Sequential(
            CBR(in_c, out_c),
            CBR(out_c, out_c),
        )

    def forward(self, x, skip):
        x = self.up(x)
        x = torch.cat([x, skip], dim=1)
        return self.conv(x)


class UNetLite(nn.Module):
    def __init__(self, n_cls: int = NUM_CLASSES, base: int = 32):
        super().__init__()
        self.d1 = Down(3, base)
        self.d2 = Down(base, base * 2)
        self.d3 = Down(base * 2, base * 4)
        self.d4 = Down(base * 4, base * 8)
        self.pool = nn.MaxPool2d(2)
        self.bridge = Down(base * 8, base * 16)

        self.u4 = Up(base * 16, base * 8)
        self.u3 = Up(base * 8, base * 4)
        self.u2 = Up(base * 4, base * 2)
        self.u1 = Up(base * 2, base)

        self.out_conv = nn.Conv2d(base, n_cls, kernel_size=1)

    def forward(self, x):
        x1 = self.d1(x)
        x2 = self.d2(self.pool(x1))
        x3 = self.d3(self.pool(x2))
        x4 = self.d4(self.pool(x3))
        x = self.bridge(self.pool(x4))
        x = self.u4(x, x4)
        x = self.u3(x, x3)
        x = self.u2(x, x2)
        x = self.u1(x, x1)
        return self.out_conv(x)


my_cnn = UNetLite().to(DEVICE)

miou_my_cnn = run_training(
    model=my_cnn,
    epochs=12,
    lr=1e-3,
)

print(f"UNet-Lite mIoU: {miou_my_cnn:.3f}")

E00: mIoU=0.039
E01: mIoU=0.039
E02: mIoU=0.040
E03: mIoU=0.043
E04: mIoU=0.045
E05: mIoU=0.045
E06: mIoU=0.045
E07: mIoU=0.043
E08: mIoU=0.044
E09: mIoU=0.042
E10: mIoU=0.044
E11: mIoU=0.049
E12: mIoU=0.051
E13: mIoU=0.052
E14: mIoU=0.052
UNet-Lite mIoU: 0.052


### Краткие выводы

- **Ограниченная выразительность:**  
  Модель без предобучения и с малым количеством параметров не справляется с задачей сложной сегментации.

- **Медленное обучение:**  
  Прирост mIoU идёт очень слабо, итоговое качество остаётся низким даже после нескольких эпох.

- **Рекомендации по улучшению:**  
  Для прогресса стоит увеличить архитектурную ёмкость — добавить больше уровней или каналов, использовать предобученные энкодеры (например, из Unet++ или SegFormer), а также применить более агрессивные аугментации.


### Улучшенная

- **Архитектура**:  
  UNet-Lite + SCSE (Channel & Spatial Squeeze-Excitation) после каждого Up-блока, базовая ширина канала = 48.  
- **Обучение**:  
  20 эпох, lr = 5 × 10⁻⁴, AdamW + LambdaLR, AMP, комбинированный loss = 0.7 × CE + 0.3 × Dice, метрика – mIoU.

In [9]:
class SCSE(nn.Module):
    def __init__(self, c: int, r: int = 16):
        super().__init__()
        self.cSE = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(c, c // r, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(c // r, c, kernel_size=1),
            nn.Sigmoid(),
        )
        self.sSE = nn.Sequential(
            nn.Conv2d(c, 1, kernel_size=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.cSE(x) + x * self.sSE(x)


class UNetLiteSCSE(UNetLite):
    def __init__(self, n_cls: int = NUM_CLASSES, base: int = 48):
        super().__init__(n_cls, base)
        self.scse4 = SCSE(base * 8)
        self.scse3 = SCSE(base * 4)
        self.scse2 = SCSE(base * 2)
        self.scse1 = SCSE(base)

    def forward(self, x):
        x1 = self.d1(x)
        x2 = self.d2(self.pool(x1))
        x3 = self.d3(self.pool(x2))
        x4 = self.d4(self.pool(x3))
        x = self.bridge(self.pool(x4))
        x = self.scse4(self.u4(x, x4))
        x = self.scse3(self.u3(x, x3))
        x = self.scse2(self.u2(x, x2))
        x = self.scse1(self.u1(x, x1))
        return self.out_conv(x)


my_cnn_imp = UNetLiteSCSE().to(DEVICE)

miou_my_cnn_imp = run_training(
    model=my_cnn_imp,
    epochs=15,
    lr=5e-4,
)

print(f"Improved-UNet-Lite mIoU: {miou_my_cnn_imp:.3f}")

E00: mIoU=0.039
E01: mIoU=0.039
E02: mIoU=0.039
E03: mIoU=0.039
E04: mIoU=0.039
E05: mIoU=0.041
E06: mIoU=0.042
E07: mIoU=0.040
E08: mIoU=0.041
E09: mIoU=0.042
E10: mIoU=0.041
E11: mIoU=0.043
E12: mIoU=0.046
E13: mIoU=0.046
E14: mIoU=0.049
E15: mIoU=0.050
E16: mIoU=0.051
E17: mIoU=0.050
E18: mIoU=0.051
E19: mIoU=0.054
Improved-UNet-Lite mIoU: 0.054


## Результаты

In [14]:
import pandas as pd

results = pd.DataFrame(
    {
        "Model": [
            "Baseline-CNN",
            "Baseline-Trans",
            "Improved-CNN",
            "Improved-Trans",
            "My-CNN",
            "My-CNN-Imp",
        ],
        "mIoU": [
            miou_base_cnn,
            miou_base_trans,
            miou_imp_cnn,
            miou_imp_trans,
            miou_my_cnn,
            miou_my_cnn_imp,
        ],
    }
)

print(results)

            Model      mIoU
0    Baseline-CNN  0.133739
1  Baseline-Trans  0.649590
2    Improved-CNN  0.257460
3  Improved-Trans  0.684180
4          My-CNN  0.051553
5      My-CNN-Imp  0.054332


Ввыводы
Добавление механизма SCSE приводит к небольшому приросту mIoU: с 0.052 до 0.054.

Несмотря на улучшение, модель остаётся слишком компактной.

Усиление внимания помогает, но для заметного повышения качества требуется более глубокая или широкая архитектура, либо использование предобученных энкодеров.

Вывод
Сегментаторы на базе трансформеров (Segformer) последовательно превосходят CNN-модели. Даже с улучшениями, CNN-архитектурам не хватает мощности — без серьёзного увеличения вычислительных возможностей и глубины они не способны конкурировать с attention-базированными решениями.

